In [183]:
import pandas as pd

In [184]:
df = pd.read_excel('data_housing_fullhisto.xlsx', skiprows=20)


In [185]:
df.head(10)

,Code,Area and rooms of apartment,Currency,Year,Average,January-March,April-June,July-September,October-December
0,51000,Total,NIS thousand,2017,1531.5,1532.1,1523.2,1537.3,1533.1
1,51010,1-2 (Total),NIS thousand,2017,1018.8,954.5,1007,1066.2,1059
2,51030,3-2.5 (Total),NIS thousand,2017,1167.2,1161.5,1147.3,1193.8,1166.2
3,51050,4-3.5 (Total),NIS thousand,2017,1504.5,1528,1472,1509.2,1507.8
4,51070,5-4.5 (Total),NIS thousand,2017,1951.5,1949.7,1951,1938.1,1967.7
5,51090,6-5.5 (Total),NIS thousand,2017,2491.7,2417.4,2524.9,2517,2522.9
6,51100,Jerusalem District - 1,NIS thousand,2017,1862,1852.5,1854.1,1884.8,1858.3
7,51110,1-2 (Jerusalem District - 1),NIS thousand,2017,1234.1,1215.1,1248.9,1225,1243.4
8,51130,3-2.5 (Jerusalem District - 1),NIS thousand,2017,1523,1501.1,1533.3,1578.3,1484.8
9,51150,4-3.5 (Jerusalem District - 1),NIS thousand,2017,1903.8,1864.2,1907.8,1915.5,1937.2


In [186]:
df.columns

Index(['Code', 'Area and rooms of apartment', 'Currency', 'Year', 'Average',
       'January-March', 'April-June', 'July-September', 'October-December'],
      dtype='object')

In [187]:
quarters_dict = {'January-March': '1Q', 'April-June': '2Q', 'July-September': '3Q', 'October-December': '4Q'}

#replace 
df.columns = df.columns.map(lambda x: quarters_dict.get(x, x))
df = df.drop(columns="Average")

#unpivot
df_unpivoted = df.melt(id_vars=['Code', 'Area and rooms of apartment', 'Currency', 'Year'], var_name='Quarter', value_name='Average Price')

In [188]:
df_unpivoted.head()

,Code,Area and rooms of apartment,Currency,Year,Quarter,Average Price
0,51000,Total,NIS thousand,2017,1Q,1532.1
1,51010,1-2 (Total),NIS thousand,2017,1Q,954.5
2,51030,3-2.5 (Total),NIS thousand,2017,1Q,1161.5
3,51050,4-3.5 (Total),NIS thousand,2017,1Q,1528
4,51070,5-4.5 (Total),NIS thousand,2017,1Q,1949.7


In [189]:
def extract_rooms(text):
    """Extract room number range (X-Y format) from apartment description."""
    import re
    match = re.search(r'(\d+(?:\.\d+)?)-(\d+(?:\.\d+)?)', str(text))
    return f"{match.group(1)}-{match.group(2)}" if match else 'All'


df_unpivoted["Rooms"] = df_unpivoted["Area and rooms of apartment"].apply(extract_rooms)



df_unpivoted.head(20)

,Code,Area and rooms of apartment,Currency,Year,Quarter,Average Price,Rooms
0,51000,Total,NIS thousand,2017,1Q,1532.1,All
1,51010,1-2 (Total),NIS thousand,2017,1Q,954.5,1-2
2,51030,3-2.5 (Total),NIS thousand,2017,1Q,1161.5,3-2.5
3,51050,4-3.5 (Total),NIS thousand,2017,1Q,1528,4-3.5
4,51070,5-4.5 (Total),NIS thousand,2017,1Q,1949.7,5-4.5
5,51090,6-5.5 (Total),NIS thousand,2017,1Q,2417.4,6-5.5
6,51100,Jerusalem District - 1,NIS thousand,2017,1Q,1852.5,All
7,51110,1-2 (Jerusalem District - 1),NIS thousand,2017,1Q,1215.1,1-2
8,51130,3-2.5 (Jerusalem District - 1),NIS thousand,2017,1Q,1501.1,3-2.5
9,51150,4-3.5 (Jerusalem District - 1),NIS thousand,2017,1Q,1864.2,4-3.5


In [190]:
def extract_area(text):
    """Extract area from apartment description, removing room numbers."""
    import re
    text = str(text)
    # Check if there's content in parentheses
    paren_match = re.search(r'\(([^)]+)\)', text)
    if paren_match:
        area = paren_match.group(1)
    else:
        area = text
    # Remove the "- X" or "- X-Y" pattern
    area = re.sub(r'\s*-\s*\d+(?:\.\d+)?(?:-\d+(?:\.\d+)?)?', '', area)

    if area.lower() == 'total':
        area = 'Israel'

    return area.strip()


df_unpivoted["Area"] = df_unpivoted["Area and rooms of apartment"].apply(extract_area)

df_unpivoted.head(20)

,Code,Area and rooms of apartment,Currency,Year,Quarter,Average Price,Rooms,Area
0,51000,Total,NIS thousand,2017,1Q,1532.1,All,Israel
1,51010,1-2 (Total),NIS thousand,2017,1Q,954.5,1-2,Israel
2,51030,3-2.5 (Total),NIS thousand,2017,1Q,1161.5,3-2.5,Israel
3,51050,4-3.5 (Total),NIS thousand,2017,1Q,1528,4-3.5,Israel
4,51070,5-4.5 (Total),NIS thousand,2017,1Q,1949.7,5-4.5,Israel
5,51090,6-5.5 (Total),NIS thousand,2017,1Q,2417.4,6-5.5,Israel
6,51100,Jerusalem District - 1,NIS thousand,2017,1Q,1852.5,All,Jerusalem District
7,51110,1-2 (Jerusalem District - 1),NIS thousand,2017,1Q,1215.1,1-2,Jerusalem District
8,51130,3-2.5 (Jerusalem District - 1),NIS thousand,2017,1Q,1501.1,3-2.5,Jerusalem District
9,51150,4-3.5 (Jerusalem District - 1),NIS thousand,2017,1Q,1864.2,4-3.5,Jerusalem District


In [191]:
df_unpivoted = df_unpivoted[~(df_unpivoted["Average Price"] == '-')]

In [192]:
df_unpivoted["Is_District"] = df_unpivoted["Area"].str.contains("District", case=False)


In [193]:
df_unpivoted = df_unpivoted.drop(columns=["Area and rooms of apartment"])
df_unpivoted["Quarter_ts"] = pd.PeriodIndex(year=df_unpivoted['Year'], quarter=df_unpivoted['Quarter'].str[0].astype(int), freq='Q').to_timestamp()
df_unpivoted = df_unpivoted[['Area', 'Rooms', 'Currency', 'Year', 'Quarter', 'Quarter_ts', 'Average Price', 'Is_District']]



C:\Users\dbenayou\AppData\Local\Temp\ipykernel_71972\3939158524.py:2: FutureWarning: Constructing PeriodIndex from fields is deprecated. Use PeriodIndex.from_fields instead.
  df_unpivoted["Quarter_ts"] = pd.PeriodIndex(year=df_unpivoted['Year'], quarter=df_unpivoted['Quarter'].str[0].astype(int), freq='Q').to_timestamp()


In [194]:
df_unpivoted.to_excel('data_housing_unpivoted.xlsx', index=False)

In [195]:
df_unpivoted.head(20)

,Area,Rooms,Currency,Year,Quarter,Quarter_ts,Average Price,Is_District
0,Israel,All,NIS thousand,2017,1Q,2017-01-01,1532.1,False
1,Israel,1-2,NIS thousand,2017,1Q,2017-01-01,954.5,False
2,Israel,3-2.5,NIS thousand,2017,1Q,2017-01-01,1161.5,False
3,Israel,4-3.5,NIS thousand,2017,1Q,2017-01-01,1528,False
4,Israel,5-4.5,NIS thousand,2017,1Q,2017-01-01,1949.7,False
5,Israel,6-5.5,NIS thousand,2017,1Q,2017-01-01,2417.4,False
6,Jerusalem District,All,NIS thousand,2017,1Q,2017-01-01,1852.5,True
7,Jerusalem District,1-2,NIS thousand,2017,1Q,2017-01-01,1215.1,True
8,Jerusalem District,3-2.5,NIS thousand,2017,1Q,2017-01-01,1501.1,True
9,Jerusalem District,4-3.5,NIS thousand,2017,1Q,2017-01-01,1864.2,True
